## CODIGO PRINCIPAL

En este notebook se incluyen las ejecuciones de los programas desarrollados en el proyecto de forma secuencial.

En concreto este código está dedicado a los capítulos del trabajo indicados a continuación:

    3. Sistema, diseño y desarrollo
    4. Experimentos realizados y resultados

#### Importar funciones y módulos requeridos

El módulo *nimgenetics.py* contiene las funciones necesarias para ejecutar este código.

In [1]:
import os
import sys

sys.path.append('./my_module/')
import nimgenetics

### 3. SISTEMA, DISEÑO Y DESARROLLO

     
Directorios importantes:
     
- folder_path: carpeta donde se ubican los informes de NIMGenetics.
- folder_result: carpeta donde se irán guardando los resultados de las ejecuciones de los programas.
- pathI: path de la carpeta "Identificados" de los informes que cumplen el patrón de selección.
- image_path: carpeta donde se almacenan las imágenes generadas a partir de los informes.
- textos: carpeta donde se almacenan los textos generados a partir de las imágenes.

In [ ]:
folder_path = 'path_to_folder_root'
folder_result = 'path_to_folder_result'
pathI = 'path_to_folder_identificados'
image_path = 'path_to_images'
textos = 'path_to_textos'

#### 3.2 Obtención de informes clínicos de Gestlab
Informes filtrados y renombrados.

#### 3.2.1 Mapeo de los informes localizados en folder_path
La función load_files es utilizada para mapear los informes (PDF) dentro de folder_path.

In [ ]:
# l_path contiene el path absoluto de todos los informes
l_path = nimgenetics.load_files(root_dir=folder_path)[0]
# l_pdf es el mapeo individual de cada informe
l_pdf = nimgenetics.load_files(root_dir=folder_path)[1]

#### 3.2.2 Obtención de los identificadores reales de los informes
La función getIDs realiza una consulta sql a Gestlab para obtener los identificadores reales.

In [ ]:
# id_peticion es una lista con los ID reales obtenidos tras la decodificación hexadecimal
id_peticion = []
for pdf in l_pdf:
    # split por guion bajo
    s_pdf = pdf.split('_')
    # transformación del segundo componente en hexadecimal del identificador
    hex_id = int(s_pdf[1],16)
    id_peticion.append(hex_id)

# ejecución de la función getIDs
r_id = nimgenetics.getIDs(id_list=id_peticion)

#### 3.2.3 Almacenamiento de los informes con su ID correcto
Selección de informes que cumplen un patrón específico: identifiedPDF.

Informes cuyo identificador es desconocido: emptyPDF.

In [ ]:
# Generar una lista que almacene todos los identificadores provenientes de r_id
# l_names a diferencia de la anterior no es una lista de listas
l_names = []   
for row in r_id:
    try:
        # Primer bloque: extraer identificadores que cumplan la siguiente estructura
        l_names.append(row[0][0])
    except:
        # Segundo bloque: identificadores que devuelven elementos vacios
        # Estos informes se almacenarán con su identificador hexadecimal original
        s_row = str(row)
        if s_row == '[]':
            # Almacenamos en l_names los identificadores reemplazados con el literal 'sin_id'
            sr_row = s_row.replace('[]', 'sin_id')
            l_names.append(sr_row)


# Función utilizada para almacenar los informes que cumplen un patrón de selección pre-establecido
# También se almacenan los informes que están repetidos y los descartados que no cumplen el patrón
# con la finalidad de no perder información
nimgenetics.identifiedPDF(folder_path=folder_result, list_id=l_names, list_path=l_path)

# Función para almacenar los informes cuyo identificador no ha sido recuperado de la base de datos
# Estos informes son guardados con el identificador hexadecimal original
nimgenetics.emptyPDF(folder_path=folder_result, list_id=l_names, list_path=l_path, list_pdf=l_pdf)

#### 3.3 Conversión de los informes a texto plano para la extracción de terminología médica

In [ ]:
# pdfPath contiene el path absoluto de todos los pdf 
# este paso es necesario para ejecutar el siguiente programa: pdfToImage
pdfPath = nimgenetics.absolutePath(dpath=pathI)

#### 3.3.1 Generación de imágenes en formato PNG a partir de los informes escaneados
Este procedimiento involucra la ejecución de la función pdfToImage.

In [ ]:
# Ejecución de la función pdfToImage
nimgenetics.pdfToImage(f_path=folder_result, pdf_list=pdfPath)

#### 3.3.2 Generación de texto plano a partir de imágenes PNG
Este procedimiento involucra la ejecución de dos funciones: makeCorpus y ocrToImage.

In [ ]:
# Ejecución de la función makeCorpus la cual integra en su código ocrToImage
# Necesario image_path como parámetro para la correcta ejecución de la función
# El resultado de este proceso guarda los resultados en la carpeta textos
nimgenetics.makeCorpus(f_path=folder_result, image_path=image_path)

### 4. EXPERIMENTOS REALIZADOS Y RESULTADOS

Directorios importantes:

- cutext_results: carpeta que incluye los resultados de la ejecución de CUTEXT.
- cutext_corpus: carpeta que alberga el corpus elaborado al procesar los resultados de CUTEXT.
- megadict: path donde se localiza el megadiccionario.
- lookup_corpus: path donde almacenar el corpus generado por la estrategia lookup.
- corpus_global: path donde guardar el corpus global elaborado a partir de cutext y lookup.
- lookup_corpus_uncleaned: path donde se almacena el corpus de lookup sin proceso de curación.

In [ ]:
cutext_results = 'path_to_save_cutext_results'
cutext_corpus = 'path_to_save_cutext_corpus'
megadict = 'path_to_megadict'
lookup_corpus = 'path_to_lookup_corpus'
corpus_global = 'path_to_merge_corpus'
lookup_corpus_uncleaned = 'path_to_lookup_corpusUncleaned'

In [ ]:
# path_textos contiene el path absoluto de los textos
path_textos = nimgenetics.absolutePath(dpath=textos)

#### 4.2 Extracción de terminología mediante CUTEXT
Ejecutar la función getCutextResults que integra la funcionalidad de CUTEXT.

In [ ]:
# path a la carpeta in de cutext
cutext_in = 'path_to_cutext_in_folder'
# path del output generado por cutext
out = 'path_to_cutext_output_folder'
# El output de cutext se almacenará en cutext_results
# ejecución de la función que integra la funcionalidad de cutext
nimgenetics.getCutextResults(cutext_in=cutext_in, textos_ori=path_textos, old=out, new=cutext_results)

#### 4.2.1 Elaboración de un corpus final a partir de los resultados CUTEXT
La función cutextCorpus procesa los resultados generados en el apartado anterior.

In [ ]:
# path absoluto de los resultados de cutext
l_cutext = nimgenetics.absolutePath(dpath=cutext_results)

# Ejecución del programa que genera el corpus de cutext
# El resultado se almacena en cutext_corpus
nimgenetics.cutextCorpus(lista_cutext=l_cutext, path_to_words=cutext_corpus)

#### 4.3 Extracción de terminología mediante estrategia lookup

La función lookup genera un corpus elaborado utilizando esta estrategia.

In [ ]:
# Almacenamos el corpus de lookup en lookup_corpus
nimgenetics.lookup(textos_ori=path_textos, megadict=megadict, path_to_words=lookup_corpus)

#### 4.5 Elaboración de un corpus global a partir de cutext y lookup
La función mergeCorpus implica la construcción de un corpus que alberga la terminología extraída mediante cutext y la estrategia lookup.

In [ ]:
# Este proceso genera el corpus global y se almacena en corpus_global
nimgenetics.mergeCorpus(p_cutext=cutext_corpus, p_lookup=lookup_corpus, path_to_words=corpus_global)

#### 4.5.1 Representación de clínica sobre una matriz de distribución terminológica
Diseñar una función capaz de mapear los términos extraídos al paciente correspondiente

In [ ]:
# Construir la matriz a partir de un dataFrame almacenado como un documento excel
nimgenetics.mapeoTerminos(l_textos=path_textos, infile_global=corpus_global)

#### 4.5.2 Selección de 500 términos para el mapeo manual a HPO
Función que genera dos diccionarios relativo al conteo global de términos y los 500 términos para el mapeo manual a HPO.

In [ ]:
all_terms = nimgenetics.subsetManual(c_uncleaned=lookup_corpus_uncleaned, c_global=corpus_global)

El programa devuelve el conteo global -*all_terms*- gracias a la función items del módulo *FreqDist* de NLTK. 

Esta lista es utilizada para generar la imagen de la distribución terminológia en función de la frecuencia de los términos.

In [ ]:
# -all_terms- diccionario que integra la lista completa de términos
# Los términos aparecen en formato key, value
# key = término
# value = número de veces que aparece el término

# Primera categoría:
values_500 = []
values_500_50 = []
values_50 = []

for key, value in all_terms:
    
    if value > 500:
        values_500.append(key)
    elif value <= 500 and value >= 50:
        values_500_50.append(key)
    else:
        values_50.append(key)

names = ['500', '500-50', '50']
values = [len(values_500), len(values_500_50), len(values_50)]

# Segunda categoría:
# Diccionario con las palabras <= 50 repeticiones en el corpus global
dict2 = {k: v for k, v in all_terms if v <= 50}
d2 = dict2.items()

v_50_25 = []
v_25_10 = []
v_10 = []

for key, value in d2:
    
    if value <= 50 and value > 25:
        v_50_25.append(key)
    elif value <= 25 and value > 10:
        v25_10.append(key)
    else:
        v_10.append(key)

names1 = ['50-25', '25-10', '10']
values1 = [len(v_50_25), len(v_25_10), len(v_10)]

# Tercera categoría:
# Diccionario con las palabras <= 10 repeticiones en el corpus global
dict3 = {k: v for k, v in all_terms if v <= 10}
d3 = dict3.items()

v_10_5 = []
v_5_3 = []
v_3 = []

for key, value in d3:
    
    if value <= 10 and value > 5:
        v_10_5.append(key)
    elif value <= 5 and value > 3:
        v_5_3.append(key)
    else:
        v_3.append(key)
        
names2 = ['10-5', '5-3', '3']
values2 = [len(v_10_5), len(v_5_3), len(v_3)]

# Cuarta categoría:
# Diccionario con las palabras <= 3 repeticiones en el corpus global
dict4 = {k: v for k, v in all_terms if v <= 3}
d4 = dict4.items()

v_3_ = []
v_2_ = []
v_1_ = []

for key, value in d4:
    
    if value == 3:
        v_3_.append(key)
    elif value == 2:
        v_2_.append(key)
    else:
        v_1_.append(key)

names3 = ['3', '2', '1']
values3 = [len(v_3_), len(v_2_), len(v_1_)]

# Plotear la distribución de categorías terminológicas
fig = plt.figure(figsize=(12,10))
ax0 = fig.add_subplot(111)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax0.spines['top'].set_color('none')
ax0.spines['bottom'].set_color('none')
ax0.spines['left'].set_color('none')
ax0.spines['right'].set_color('none')
ax0.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

ax0.set_ylabel('Conteo de términos', fontsize=12, labelpad=20)
ax0.set_xlabel('Frecuencia de términos', fontsize=12, labelpad=20)

ax1.bar(names, values)
ax1.set_ylim(0, 4500)

ax2.bar(names1, values1)
ax2.set_ylim(0, 4500)

ax3.bar(names2, values2)
ax3.set_ylim(0, 4500)

ax4.bar(names3, values3)
ax4.set_ylim(0, 4500)

fig.suptitle('Categorias Terminológicas', fontsize=20)
fig.savefig('distribucion de terminología.png')

### 4.6 EXOMISER

Función que genera el template yml para poder ejecutar exomiser.

In [ ]:
# path para el archivo yml y para guardar los resultados de la ejecución
infile_template = 'path_to_yml_template'
exo_path = 'path_to_save_exomiser_results'

# Hacer el diccionario con los datos
d = {'16NS-1': ['HP:0002715','HP:0002086'],
     '16NS-2': ['HP:0002145','HP:0001251','HP:0002072','HP:0001260','HP:0002015','HP:0002127','HP:0002071',
                'HP:0001272','HP:0002145','HP:0002527','HP:0002463','HP:0000666'],
     '16NS-3': ['HP:0001644'],
     '17NR-1': ['HP:0000729','HP:0001999','HP:0000286','HP:0000414','HP:0001611','HP:0000712'],
     '17NR-2': ['HP:0002352','HP:0003429','HP:0002415','HP:0001252','HP:0012736','HP:0000577','HP:0030455',
                'HP:0000338','HP:0000817','HP:0001762','HP:0002808','HP:0003376'],
     '17NR-3': ['HP:0001263','HP:0007018','HP:0006919'],
     '17NR-4': ['HP:0000118','HP:0000252','HP:0000532','HP:0001004','HP:0001363','HP:0010537','HP:0000233',
                'HP:0000343','HP:0000581','HP:0025349','HP:0000488','HP:0001622'],
     '18NR-1': ['HP:0000158','HP:0011410','HP:0002804','HP:0001298','HP:0001250','HP:0001274','HP:0001166',
                'HP:0000119','HP:0001197'],
     '18NR-2': ['HP:0004322','HP:0100021','HP:0001332','HP:0001263','HP:0001510','HP:0001250'],
     '18NR-3': ['HP:0000924','HP:0000152','HP:0001507','HP:0040064','HP:0000707','HP:0000598','HP:0001574',
                'HP:0001626','HP:0002715','HP:0002086','HP:0003011','HP:0001263','HP:0000403']}

keys = list(d.keys())
values = list(d.values())

# ejecutar la función para preparar los templates para ejecutar exomiser
nimgenetics.makeTemplate(patient_id=keys, lista_hpo=values, infile=infile_template, exomiser_path=exo_path)

#### 4.6.2 Resultados de la ejecución de exomiser y comparativa con Health29

Lista de variantes candidatas obtenida a partir de la ejecución de exomiser.

#### 4.6.2.2 Caso 16NS-2

Análisis del output del caso 16NS-2.

In [2]:
caso2 = 'path_to_16NS-2_output.variants.tsv'
nimgenetics.filterVariants(data=caso2)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chrX,56591569,missense_variant,UBQLN2,0.996715,0.787443,0.996715,0.982886
1,chr14,92537354,frameshift_elongation,ATXN3,0.950000,0.706157,0.950000,0.941401
2,chr16,730628,missense_variant,STUB1,1.000000,0.655719,1.000000,0.937705
4,chr16,732171,frameshift_truncation,STUB1,0.950000,0.655719,1.000000,0.937705
5,chr7,73795169,missense_variant,CLIP2,0.987958,0.657653,0.987958,0.932205
6,chr9,135218103,missense_variant,SETX,0.881138,0.749208,0.881138,0.930327
7,chr6,170871049,disruptive_inframe_insertion,TBP,0.850000,0.764932,0.850000,0.921954
9,chr22,29886606,missense_variant,NEFH,0.997507,0.614849,0.997507,0.905837
10,chr9,140773612,splice_donor_variant,CACNA1B,0.900000,0.605219,1.000000,0.899043
11,chr9,140777306,missense_variant,CACNA1B,1.000000,0.605219,1.000000,0.899043


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [3]:
nimgenetics.locVariant(data=caso2, gen_sel='UBQLN2')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chrX,56591569,missense_variant,UBQLN2,0.996715,0.787443,0.996715,0.982886


#### 4.6.2.3 Caso 16NS-3

Análisis del output del caso 16NS-3.

In [4]:
caso3 = 'path_to_16NS-3_output.variants.tsv'
nimgenetics.filterVariants(data=caso3)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
1,chr14,23899016,missense_variant,MYH7,1.000000,1.000000,1.000000,0.998149
2,chr10,69881820,missense_variant,MYPN,0.999596,1.000000,0.999596,0.998142
4,chr2,179600304,missense_variant,TTN,0.992592,1.000000,0.992592,0.998019
6,chr2,179640548,missense_variant,TTN,0.987838,1.000000,0.992592,0.998019
7,chrX,31747837,missense_variant,DMD,0.922496,1.000000,0.922496,0.996236
9,chr5,137222922,missense_variant,MYOT,0.997149,0.829720,0.997149,0.988948
10,chr4,996513,splice_region_variant,IDUA,0.900000,0.766280,0.900000,0.949903
11,chr6,110064448,missense_variant,FIG4,0.999797,0.671483,0.999797,0.946524
15,chr6,31238930,missense_variant,HLA-B,0.758000,0.641241,0.997000,0.926458
16,chr6,31239006,missense_variant,HLA-B,0.997000,0.641241,0.997000,0.926458


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [5]:
nimgenetics.locVariant(data=caso3, gen_sel='MYH7')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr14,23892645,coding_transcript_intron_variant,MYH7,0.0,1.0,1.0,0.998149
1,chr14,23899016,missense_variant,MYH7,1.0,1.0,1.0,0.998149


#### 4.6.2.4 Caso 17NR-1

Análisis del output del caso 17NR-1.

In [6]:
caso4 = 'path_to_17NR-1_output.variants.tsv'
nimgenetics.filterVariants(data=caso4)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
162,chr7,73477660,missense_variant,ELN,0.994485,0.735234,0.994485,0.970328
163,chr22,24109722,missense_variant,CHCHD10,0.912603,0.757119,0.912603,0.950871
164,chr11,65978677,missense_variant,PACS1,0.998427,0.680921,0.998427,0.950691
166,chr16,49672716,missense_variant,ZNF423,0.998423,0.651352,0.998423,0.934115
167,chr18,48581231,missense_variant,SMAD4,0.997270,0.643719,0.997270,0.928359
168,chr22,51121852,splice_region_variant,SHANK3,0.811295,0.788879,0.811295,0.913936
169,chr12,112036767,disruptive_inframe_insertion,ATXN2,0.845490,0.757119,0.845490,0.912661
170,chr14,73659482,missense_variant,PSEN1,1.000000,0.615655,1.000000,0.908472
171,chr6,32026164,missense_variant,TNXB,0.992000,0.622534,0.992000,0.908308
173,chr1,103496754,missense_variant,COL11A1,0.991646,0.622644,0.991646,0.908132


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [7]:
nimgenetics.locVariant(data=caso4, gen_sel='PACS1')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
164,chr11,65978677,missense_variant,PACS1,0.998427,0.680921,0.998427,0.950691
165,chr11,66010762,3_prime_UTR_exon_variant,PACS1,0.000000,0.680921,0.998427,0.950691


#### 4.6.2.5 Caso 17NR-2

Análisis del output del caso 17NR-2.

In [8]:
caso5 = 'path_to_17NR-2_output.variants.tsv'
nimgenetics.filterVariants(data=caso5)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr3,132384868,missense_variant,UBA5,1.000000,0.656829,1.000000,0.938376
1,chr3,132394747,missense_variant,UBA5,0.888318,0.656829,1.000000,0.938376
2,chr12,7361645,missense_variant,PEX5,0.983341,0.653448,0.983341,0.926557
3,chr5,60368955,missense_variant,NDUFAF2,0.994657,0.634968,0.994657,0.920326
4,chr19,39911625,missense_variant,PLEKHG2,0.992780,0.631986,0.992780,0.916717
5,chr4,3076650,frameshift_truncation,HTT,0.950000,0.633110,0.950000,0.882608
6,chr4,3076653,frameshift_variant,HTT,0.950000,0.633110,0.950000,0.882608
7,chr4,3076659,missense_variant,HTT,0.785737,0.633110,0.950000,0.882608
8,chr4,3076662,missense_variant,HTT,0.759000,0.633110,0.950000,0.882608
9,chr4,3076665,missense_variant,HTT,0.644328,0.633110,0.950000,0.882608


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [9]:
nimgenetics.locVariant(data=caso5, gen_sel='UBA5')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr3,132384868,missense_variant,UBA5,1.000000,0.656829,1.0,0.938376
1,chr3,132394747,missense_variant,UBA5,0.888318,0.656829,1.0,0.938376


#### 4.6.2.6 Caso 17NR-3

Análisis del output del caso 17NR-3.

In [10]:
caso6 = 'path_to_17NR-3_output.variants.tsv'
nimgenetics.filterVariants(data=caso6)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
34,chr3,9517375,missense_variant,SETD5,0.931024,0.904502,0.931024,0.990660
224,chr7,73482987,missense_variant,ELN,0.902591,0.895430,0.902591,0.986727
225,chr6,170871051,frameshift_truncation,TBP,0.950000,0.819662,0.950000,0.981229
226,chr6,170871054,frameshift_variant,TBP,0.950000,0.819662,0.950000,0.981229
228,chr1,151400625,stop_gained,POGZ,0.950000,0.817993,0.950000,0.980906
229,chr16,2112607,splice_region_variant,TSC2,0.900000,0.787658,0.900000,0.959481
230,chr9,140773612,splice_donor_variant,CACNA1B,0.900000,0.650247,1.000000,0.934299
231,chr9,140777306,missense_variant,CACNA1B,1.000000,0.650247,1.000000,0.934299
233,chr4,23816235,splice_region_variant,PPARGC1A,0.900000,0.728623,0.900000,0.927639
234,chr1,110256115,missense_variant,GSTM5,0.994000,0.641622,0.994000,0.924835


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [11]:
nimgenetics.locVariant(data=caso6, gen_sel='POGZ')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
227,chr1,151384338,coding_transcript_intron_variant,POGZ,0.00,0.817993,0.95,0.980906
228,chr1,151400625,stop_gained,POGZ,0.95,0.817993,0.95,0.980906


#### 4.6.2.7 Caso 17NR-4

Análisis del output del caso 17NR-4.

In [12]:
caso7 = 'path_to_17NR-4_output.variants.tsv'
nimgenetics.filterVariants(data=caso7)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr10,97402858,missense_variant,ALDH18A1,0.984244,0.645947,0.984244,0.921672
169,chr10,94410275,splice_donor_variant,KIF11,0.900000,0.704208,0.900000,0.908645
174,chr6,31239006,missense_variant,HLA-B,0.997000,0.577401,0.997000,0.866450
177,chr6,31323202,missense_variant,HLA-B,0.001000,0.577401,0.997000,0.866450
178,chr6,31323962,frameshift_elongation,HLA-B,0.950000,0.577401,0.997000,0.866450
179,chr6,31324100,missense_variant,HLA-B,0.488000,0.577401,0.997000,0.866450
181,chr1,120510194,missense_variant,NOTCH2,0.835154,0.625004,0.942000,0.865254
183,chr1,120572547,missense_variant,NOTCH2,0.871000,0.625004,0.942000,0.865254
184,chr1,120572572,missense_variant,NOTCH2,0.942000,0.625004,0.942000,0.865254
193,chr1,120611964,missense_variant,NOTCH2,0.815000,0.625004,0.942000,0.865254


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [13]:
nimgenetics.locVariant(data=caso7, gen_sel='KIF11')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
169,chr10,94410275,splice_donor_variant,KIF11,0.9,0.704208,0.9,0.908645


#### Caso 18NR-1

Análisis del output del caso 18NR-1.

In [14]:
caso8 = 'path_to_18NR-1_output.variants.tsv'
nimgenetics.filterVariants(data=caso8)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr18,42531907,missense_variant,SETBP1,1.000000,0.673894,1.000000,0.947870
1,chr1,8420514,missense_variant,RERE,1.000000,0.622745,1.000000,0.914418
5,chr1,6111709,missense_variant,KCNAB2,0.981009,0.622745,0.981009,0.899745
7,chrX,132888207,splice_region_variant,GPC3,0.900000,0.693116,0.900000,0.898613
9,chr2,152420160,missense_variant,NEB,0.717769,0.626933,0.959208,0.884700
11,chr2,152584282,missense_variant,NEB,0.959208,0.626933,0.959208,0.884700
12,chr6,3225770,missense_variant,TUBB2B,1.000000,0.589064,1.000000,0.882748
13,chr22,21153431,missense_variant,PI4KA,0.982848,0.593527,0.982848,0.870747
14,chr10,101487264,missense_variant,COX15,0.980254,0.595636,0.980254,0.870533
16,chr6,142691950,frameshift_truncation,ADGRG6,0.950000,0.619586,0.950000,0.867242


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [15]:
nimgenetics.locVariant(data=caso8, gen_sel='SETBP1')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr18,42531907,missense_variant,SETBP1,1.0,0.673894,1.0,0.94787


#### Caso 18NR-2

Análisis del output del caso 18NR-2.

In [16]:
caso9 = 'path_to_18NR-2_output.variants.tsv'
nimgenetics.filterVariants(data=caso9)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr19,6495816,missense_variant,TUBB4A,0.999295,0.765441,0.999295,0.979074
1,chr9,140777306,missense_variant,CACNA1B,1.000000,0.737432,1.000000,0.972373
3,chr9,141016163,missense_variant,CACNA1B,0.994195,0.737432,1.000000,0.972373
4,chr22,24167605,splice_region_variant,SMARCB1,0.900000,0.817198,0.900000,0.969871
5,chr6,170871051,frameshift_truncation,TBP,0.950000,0.713726,0.950000,0.945593
6,chr6,170871054,frameshift_truncation,TBP,0.950000,0.713726,0.950000,0.945593
8,chr11,26621221,missense_variant,ANO3,0.999983,0.656656,0.999983,0.938263
10,chr19,50364706,missense_variant,PNKP,0.998984,0.655126,0.998984,0.936794
11,chr6,33419613,frameshift_truncation,SYNGAP1,0.950000,0.692028,0.950000,0.932754
12,chr6,33419615,frameshift_elongation,SYNGAP1,0.950000,0.692028,0.950000,0.932754


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [17]:
nimgenetics.locVariant(data=caso9, gen_sel='ANO3')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
7,chr11,26581382,coding_transcript_intron_variant,ANO3,0.000000,0.656656,0.999983,0.938263
8,chr11,26621221,missense_variant,ANO3,0.999983,0.656656,0.999983,0.938263
9,chr11,26645955,coding_transcript_intron_variant,ANO3,0.000000,0.656656,0.999983,0.938263


#### Caso 18NR-3

Análisis del output del caso 18NR-3.

In [18]:
caso10 = 'path_to_18NR-3_output.variants.tsv'
nimgenetics.filterVariants(data=caso10)

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
1,chr4,1806119,missense_variant,FGFR3,1.000000,1.0,1.000000,0.998149
3,chr7,27140845,missense_variant,HOXA2,0.999999,1.0,0.999999,0.998149
185,chr1,235894211,missense_variant,LYST,0.997261,1.0,0.997261,0.998102
187,chr6,31239006,missense_variant,HLA-B,0.997000,1.0,0.997000,0.998097
188,chr6,31239111,frameshift_elongation,HLA-B,0.950000,1.0,0.997000,0.998097
189,chr6,31239113,frameshift_truncation,HLA-B,0.950000,1.0,0.997000,0.998097
190,chr6,31239133,splice_region_variant,HLA-B,0.776014,1.0,0.997000,0.998097
191,chr6,31323115,missense_variant,HLA-B,0.723000,1.0,0.997000,0.998097
193,chr6,31324100,missense_variant,HLA-B,0.488000,1.0,0.997000,0.998097
195,chr4,151829506,missense_variant,LRBA,0.981000,1.0,0.981000,0.997797


Posición de la variante genética indicada por los analistas en la lista de genes generada por exomiser.

In [19]:
nimgenetics.locVariant(data=caso10, gen_sel='FGFR3')

,chr,pos,functional_class,gene,variant_score,pheno_score,gene_variant_score,gene_combined_score
0,chr4,1800914,coding_transcript_intron_variant,FGFR3,0.0,1.0,1.0,0.998149
1,chr4,1806119,missense_variant,FGFR3,1.0,1.0,1.0,0.998149
2,chr4,1808495,coding_transcript_intron_variant,FGFR3,0.0,1.0,1.0,0.998149
